In [ ]:
from scipy.signal import butter, filtfilt, find_peaks, freqz, sosfilt
import glob
import os
from fnmatch import fnmatch
import h5py
import matplotlib
%matplotlib widget
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd 
import sys 
import multiprocessing
from multiprocessing import Pool

In [ ]:
#mpl.use('Qt5Agg')
#plt.switch_backend('Qt5Agg')
%matplotlib ipympl
plt.plot(range(10))

In [ ]:
class avgSpec:
    def __init__(self,
            fileList,
            dataDir):

        self.fileList = fileList
        self.dataDir = dataDir
        
        
        
        

    def mProcAvg(self, fileList, dataDir):
        
        """
        Takes list of .h5 file names and directory, 
        returns averaged spectra (linear FFT units)
        
        Parameters:
            fileList (list of strings): list of file names to be averaged 
            dataDir (str): Location of files. Realitive path.
        
        Returns:
            offData (1D np arr): averaged off spectrum
            onData (1d np arr): averaged on spectrum
        
        Calls:
            avgMesData()
        """
        self.totalNumAvg = 0

        if __name__ == '__main__':
            pool = Pool(processes=24)
            self.dataList = pool.map(self.avgMesData, fileList)
            print('done')

            #Calculate total number of FFTs in returned spectra. Each data 
            #file returns number of FFTs in dataList[data file index][2]  
            for runningAverages in self.dataList:
                self.totalNumAvg += runningAverages[2]
            
        

    
    def avgMesData(self, aFile):

        #init some parameters
        offData = np.zeros(2**23)
        onData = np.zeros(2**23)
        numDataSets = 0
        numAvgInDataFile = 0
        


        #itterate through file list and open h5 files in 'r(ead)' mode
        print('ON FILE: ' + str(aFile))
        dataBin = h5py.File(self.dataDir + aFile, 'r') #makes pointer to h5
        numAvgInMesDataSet = int(dataBin.attrs['averages']) #total number of average for dataset
        allKeys = [aKey for aKey in dataBin.keys()] #key is measdata. 16 per dataset

        for aKey in allKeys[:]:
            #print('ON KEY ' + str(aKey))
            dataset = pd.read_hdf(dataDir + aFile, key = aKey) #dataframe of ROACH data (check)
        # need to print and look at this
            offData +=  np.asarray(dataset[dataset.keys()[0]]) #term or bicon
            onData += np.asarray(dataset[dataset.keys()[1]]) #the other one
            numDataSets += 1
            numAvgInDataFile += numAvgInMesDataSet

        offData /= numDataSets
        onData /= numDataSets
        
        return offData, onData, numAvgInDataFile

In [ ]:
%matplotlib 
plt.plot(10*np.log10(2 * 1000 * spec.dataList[0][0][1:]/ 2**48 / 50))
plt.plot(10*np.log10(2 * 1000 * spec.dataList[0][1][1:]/ 2**48 / 50))



In [ ]:
def mkFileList(dataDir):

	#get list of files in dataDir, pack data*.h5 named files into fileList
	dataDirContents = os.listdir(dataDir)
	fileList = [file for file in dataDirContents if fnmatch(file, 'data*.h5')]
	fileList.sort()
	return fileList
    
dataDir = './data_7_7_22_gitignore/'
fileList = mkFileList(dataDir)
spec = avgSpec(fileList, dataDir)

#spec.avgMesData('data_99.h5')
#spec.avgMesData('data_98.h5')
spec.mProcAvg(fileList[0:3], dataDir)



In [ ]:
spec.mProcAvg(fileList[0:60], dataDir)

In [ ]:
spec.mProcAvg(fileList, dataDir)

In [ ]:
def mkFileList(dataDir):

	#get list of files in dataDir, pack data*.h5 named files into fileList
	dataDirContents = os.listdir(dataDir)
	fileList = [file for file in dataDirContents if fnmatch(file, 'data*.h5')]
	fileList.sort()
	return fileList

def getData(aFile):
	dataDir = './data_7_7_22_gitignore/'
	#init some parameters
	offData = np.zeros(2**23)
	onData = np.zeros(2**23)
	totalSets = 0
	totalAverages = 0


	#itterate through file list and open h5 files in 'r(ead)' mode
	print('ON FILE: ' + str(aFile))
	dataBin = h5py.File(dataDir + aFile, 'r') #makes pointer to h5
	fileAverages = int(dataBin.attrs['averages']) #total number of average for dataset
	allKeys = [aKey for aKey in dataBin.keys()] #key is measdata. 16 per dataset

	for aKey in allKeys[:]:
		print('ON KEY ' + str(aKey))
		dataset = pd.read_hdf(dataDir + aFile, key = aKey) #dataframe of ROACH data (check)
    # need to print and look at this
		offData +=  np.asarray(dataset[dataset.keys()[0]]) #term or bicon
		onData += np.asarray(dataset[dataset.keys()[1]]) #the other one
		totalSets += 1
		totalAverages += fileAverages

	#signal[dBm] = mag^2 [fft units]
	# signal = 10log10(2[preserve power] * data[ADC units] * len(FFT)^2 * 50[Ohms] 1000[mW/W] 
	#also! compute average. Data is sum from prev. line, divide by total sets
	#it is actually faster to do this all in one go because this function is IO limited
	#BUT! there is more averaging that needs to be done so don't go to dBm
	
	#offData = 10.*np.log10(2. * offData / totalSets  / (2**48 * 50. / 1000.))
	#onData = 10.*np.log10(2. * onData / totalSets  / (2**48 * 50. / 1000.))

	offData /= totalSets
	onData /= totalSets
	
	return offData, onData, totalAverages

	
def getDataBen(dataDir):
	fileList = sorted(
		[aFile for aFile in glob.glob(dataDir + 'data*.h5')], 
		key=lambda x: int(x[x.index('data_') + 5: x.index('.h5')])
		)
	print(fileList)



	offData = np.zeros(2**23)
	onData = np.zeros(2**23)

	totalSets = 0
	totalAverages = 0
	for aFile in fileList[:100]:
		print('ON FILE: ' + str(aFile))
		dataBin = h5py.File(aFile, 'r')
		fileAverages = int(dataBin.attrs['averages'])
		allKeys = [aKey for aKey in dataBin.keys()]

		for aKey in allKeys[:]:
			print('ON KEY ' + str(aKey))
			dataset = pd.read_hdf(aFile, key = aKey)

			offData = offData + np.asarray(dataset[dataset.keys()[0]])
			onData = onData + np.asarray(dataset[dataset.keys()[1]])
			totalSets += 1
			totalAverages += fileAverages
	offData = 10.*np.log10(2. * offData / totalSets  / (2**48 * 50. / 1000.))
	onData = 10.*np.log10(2. * onData / totalSets  / (2**48 * 50. / 1000.))


	return offData, onData, totalAverages

def plotTermSigData(fileNameTerm, avgDataBicon, freqs):
	offDataTerm = np.zeros(2**23)
	onDataTerm = np.zeros(2**23)

	totalSets = 0
	print('ON FILE: ' + str(fileNameTerm))
	dataBin = h5py.File(fileNameTerm, 'r')
	allKeys = [aKey for aKey in dataBin.keys()]

	for aKey in allKeys[:]:
		print('ON KEY ' + str(aKey))
		dataset = pd.read_hdf(fileNameTerm, key = aKey)

		offDataTerm = offDataTerm + np.asarray(dataset[dataset.keys()[0]])
		onDataTerm = onDataTerm + np.asarray(dataset[dataset.keys()[1]])
		totalSets += 1

	offDataTerm = 10.*np.log10(2. * offDataTerm / totalSets  / (2**48 * 50. / 1000.))
	onDataTerm = 10.*np.log10(2. * onDataTerm / totalSets  / (2**48 * 50. / 1000.))
	peakIndices = find_peaks(avgDataBicon[1:] - onDataTerm[1:], threshold = 1.75)[0]
	peakVals = np.asarray([(avgDataBicon[1:] - onDataTerm[1:])[x] for x in peakIndices]) 

	plt.xlabel('Frequency (MHz)', labelpad = 15, **label_font)
	plt.ylabel('Ratio (dB)', **label_font)
	#plt.plot(freqs[1:], onDataTerm[1:], alpha = 0.7, label = 'Bicon Sig Gen Off')
	plt.plot(avgDataBicon[1:] - onDataTerm[1:], alpha = 0.7, label = 'Bicon Sig Gen On')
	plt.plot(peakIndices, peakVals, 'r*')

	plt.title('Ratio Bicon, Injected 123.456MHz vs No Injection')
	plt.show()
	#plt.legend(prop = legend_font)
	return peakIndices

In [ ]:
dataDir = './data_7_7_22_gitignore/'
fileList = mkFileList(dataDir)
#print(fileList)

if __name__ == '__main__':
    pool = Pool(processes=24)
    data = pool.map(getData, fileList[0:8])
    #pid, offData, onData, totalAverages = data

In [ ]:
print(np.shape(data))

In [ ]:
data

In [ ]:
termData